# Cramér-Rao lower bounds and Monte Carlo experiments

This notebook complements **Section IV: Performance references** and **Section V: Numerical results** of our paper [\[1\]][1]. Here, we implement 1) the computation of the CRLBs for an unwrapped model (see Section IV-A), and 2) the basic machinery to run Monte Carlo experiments with our proposed estimation approaches (see their implementation in [estimation_strategies_with_examples.ipynb](estimation_strategies_with_examples.ipynb)) with respect to the different parameters of interest in the clock synchronization and ranging problem. 

## Index
<a id="index"></a>

1. [Cramér-Rao lower bounds](#CRLB) (CRLB) for the unwrapped model

2. [Generic Monte Carlo experiments](#monte_carlo) function 


[1]: http://arxiv.org/abs/1906.08208

## References

[\[1\]][1]: Pol del Aguila Pla, Lissy Pellaco, Satyam Dwivedi, Peter Händel and Joakim Jaldén, "Clock synchronization over networks — Identifiability of the sawtooth model", Preprint, submitted to _IEEE Transactions on Control Systems Technology_, 2019.

## Imports, configuration and flags

In [1]:
run_examples = False
%run estimation_strategies_with_examples.ipynb
import multiprocessing as mp

## Cramér-Rao lower bounds (CRLB) for the unwrapped model
<a id="CRLB"></a>
[Back to index](#index)

In [\[1\]][1], we present the CRLBs for a linearized version of the sawtooth model. 


[1]: http://arxiv.org/abs/1906.08208


In [2]:
# Compute inverse Fisher information matrices for several values of the the involved parameters
# Attention! If one parameter is an array all the others have to be broadcastable to it
def compute_fisherinv_matrices( nrof_samples, sigma_0, sigma_1, sigma_2, beta ):
    # Compute the number of parameter combinations for which to calculate the CRLB
    nrof_combinations = np.max( [np.array( nrof_samples ).size, 
                                 np.array( sigma_0      ).size,
                                 np.array( sigma_1      ).size,
                                 np.array( sigma_2      ).size,
                                 np.array( beta         ).size] )
    # Compute total noise power 
    sigma_squared = sigma_0**2 + (sigma_1 + beta * sigma_2)**2
    # Compute ratio of noise powers
    sigma_ratio = sigma_2**2 * (sigma_1 + beta * sigma_2)**2 / sigma_squared
    # Compute common factors for all elements of the matrix
    pre_factor = ( sigma_squared / nrof_samples ) / (
                 ( nrof_samples+1 )/12 + 2/(nrof_samples-1) * sigma_ratio ) 
    # Compute relative weights in the matrix
    Iinv = np.array( [ [ (2*nrof_samples - 1)/6 + 2/(nrof_samples-1) * sigma_ratio,
                          -.5*np.ones( nrof_combinations ) ],
                       [  -.5*np.ones( nrof_combinations ), 
                         1/(nrof_samples - 1) * np.ones( nrof_combinations ) ] ] )
    # Return inverse Fisher information matrices
    Iinv = pre_factor * Iinv
    Iinv = np.moveaxis( Iinv, 2, 0 )
    return Iinv

def CRLB( f_d, phase_s, SNR_out, SNR_in, nrof_samples = 500, T_sampling = 1e-4, T_m = 1e-8 ):
    
    # Speed of light
    c = 3e8

    # Clock periods and phases
    T_s = T_m / (T_m * f_d + 1)
    # Get noise variances
    w_std = np.abs( T_s ) * 10**( - SNR_out / 20 )
    v_std = 10**( - SNR_in / 20 )
    # Sampling factor
    sampling_factor = T_sampling  / T_m
    
    # Parameters of the linearized model with slope-dependent noise
    sigma_0 = w_std
    sigma_1 = T_m * v_std
    sigma_2 = v_std / sampling_factor
    beta    = - T_s * sampling_factor * T_m * f_d

    # Compute gradients
    # Gradient for f_d
    gradient_f_d = - np.array( 
             [ [ np.zeros_like( T_s ) ], [ np.ones_like( T_s ) ] ] ) / ( T_s**2 * sampling_factor )
    # If T_s has several values, arrange so that @ (np.matmul) works coherently
    if len( gradient_f_d.shape ) == 3:
        gradient_f_d = np.moveaxis( gradient_f_d, 2, 0 )
    # Gradient for transmission delay
    gradient_trans_delay = 2 * np.array(
             [ [ np.ones_like( phase_s ) ], [ ( phase_s / ( 2 * np.pi ) - 1 ) / sampling_factor ] ] )
    # If phase_s has several values, arrange so that @ (np.matmul) works coherently
    if len( gradient_trans_delay ) == 3:
        gradient_trans_delay = np.moveaxis( gradient_trans_delay, 2, 0 )
    # Gradient for phase_s
    gradient_s_phase = - 2 * np.pi * np.array( 
             [ [ np.ones_like( phase_s ) / T_s ], [ ( phase_s / (2 * np.pi) - 1 ) / (sampling_factor * T_s) ] ] )
    # If phase_s or T_s (or both) have several values, arrange so that @ (np.matmul) works coherently
    if len( gradient_s_phase.shape ) == 3:
        gradient_s_phase = np.moveaxis( gradient_s_phase, 2, 0 )
    
    # Compute matrices
    Iinv = compute_fisherinv_matrices( nrof_samples, sigma_0, sigma_1, sigma_2, beta )

    # Compute CRLBs (flatten to get collection index (if any) in the first (only) axis)
    # Function that transposes a collection of gradients coherently with matmul
    def transpose_well( grad ):
        return np.moveaxis( grad, -1, -2 )
    # Get CRLB for f_d
    CRLB_f_d = ( transpose_well( gradient_f_d ) 
                             @ Iinv @ 
                             gradient_f_d ).flatten( )
    # Get CRLB for transmission delay
    CRLB_trans_delay = ( transpose_well( gradient_trans_delay ) 
                         @ Iinv @ 
                         gradient_trans_delay ).flatten( )
    # scale to get CRLB for rho
    CRLB_rho = ( c/2 )**2 * CRLB_trans_delay
    # Get CRLB for phase_s
    CRLB_s_phase = ( transpose_well( gradient_s_phase ) 
                     @ Iinv @ 
                     gradient_s_phase ).flatten( )
    
    return ( CRLB_f_d, CRLB_rho, CRLB_s_phase )

## Generic Monte Carlo experiments function
<a id="monte_carlo"></a>
[Back to index](#index)

This generic Monte Carlo experiments function streamlines our experimental results in [experimental_results.ipynb](https://nbviewer.jupyter.org/github/poldap/clock_sync_and_range/blob/master/empirical_results.ipynb) by implementing the mechanics of running a number of repetitions, randomizing the parameters in their default range if required, and automatically shifting through a provided grid of parameters.

In [3]:
# Helper function to get the parameters given a grid point 
# (could be more elegant, but it won't impact performance)
def get_pars( grid_point, rho, rho_range, f_d, f_d_range, phase_s, phase_s_range, nrof_samples, SNR_out, v_std,
              randomized ):
    # Parameters that can be randomized
    # If the range is not the grid
    if np.array( rho ).size == 1:
        # Random ranges
        if randomized:
             rho_now = rho_range[0] + rho_range[1] * np.random.rand( )
        # Constant range
        else:
             rho_now = rho
    # if the range is the grid
    else:
        rho_now = rho[grid_point]
    # If the frequency is not the grid
    if np.array( f_d ).size == 1:
        # Random frequency
        if randomized:
            f_d_now = f_d_range[0] + f_d_range[1] * np.random.rand( )
            if ((f_d_now<0) and (f_d_now>= -10)):
                f_d_now = -10 - 190 * np.random.rand( )
            else:
                if ((f_d_now>0)  and (f_d_now<=10)):
                    f_d_now = 10 + 190 * np.random.rand( )
        # Constant frequency
        else:
            f_d_now = f_d
    # if the frequency is the grid
    else:
        f_d_now = f_d[grid_point]
    # If the phase is not the grid
    if np.array( phase_s ).size == 1:
        # Random phase
        if randomized:
             phase_s_now = phase_s_range[0] + phase_s_range[1] * np.random.rand( )
        # Constant phase
        else:
             phase_s_now = phase_s
    # if the phase is the grid
    else:
        phase_s_now = phase_s[grid_point]
    # Parameters that can not be randomized
    # If the sample size is not the grid
    if np.array( nrof_samples ).size == 1:
        # Constant number of samples
        nrof_samples_now = nrof_samples
    # if the number of samples is the grid
    else:
        nrof_samples_now = nrof_samples[grid_point]
    # If the outer SNR is not the grid
    if np.array( SNR_out ).size == 1:
        # Constant outer SNR
        SNR_out_now = SNR_out
    # if the outer SNR is the grid
    else:
        SNR_out_now = SNR_out[grid_point]
    # If the inner SNR is not the grid
    if np.array( v_std ).size == 1:
        v_std_now = v_std
    # if the inner SNR is the grid
    else:
        v_std_now = v_std[grid_point]

    return ( rho_now, f_d_now, phase_s_now, nrof_samples_now, SNR_out_now, v_std_now )

# Helper function to run a single Monte Carlo experiment
def experiment( seed, grid_point, rho, rho_range, f_d, f_d_range, phase_s, phase_s_range,
                nrof_samples, SNR_out, v_std, T_m, T_sampling, delta_0, estimators, randomized ):
    # Create own pointer to psi (can not be pickled)
    pointer_to_psi_function = (lambda beta: psi_function( beta, T_sampling, T_m ) )
    # Seed distinctly (repeatable but different per rep and gridpoint)
    np.random.seed( seed )
    #  Get physical parameters for this repetition
    ( rho_now, f_d_now, phase_s_now, nrof_samples_now, 
      SNR_out_now, v_std_now ) = get_pars( grid_point, 
      rho, rho_range, f_d, f_d_range, phase_s, phase_s_range,
      nrof_samples, SNR_out, v_std, randomized )
    # Compute T_s from T_m and f_d
    T_s = T_m / (1 + T_m * f_d_now )
    # Get standard parameters, i.e., offset, frequency, phase and amplitude
    alpha, beta, gamma, psi = to_standard_parameters( T_sampling, delta_0, 
                               rho_now, f_d_now, T_s, phase_s_now )
    # Get noise power (depends [very slight changes with f_d] on psi)
    w_std_now = np.abs( psi ) * 10**( - SNR_out_now / 20 )
    # Generate signal with the current parameters
    y = generate_signal( nrof_samples = nrof_samples_now, beta = beta, gamma = gamma, alpha = alpha,
                         psi = psi, w_std = w_std_now, v_std = v_std_now )
    # Initialize space to return results
    b_error = np.zeros( len( estimators ) )
    g_error = np.zeros_like( b_error )
    a_error = np.zeros_like( b_error )
    f_error = np.zeros_like( b_error )
    r_error = np.zeros_like( b_error )
    p_error = np.zeros_like( b_error )
    # Estimate with the different estimators
    for estimator_index in range( len( estimators ) ):
        # Estimate
        alpha_hat, beta_hat, gamma_hat = estimators[estimator_index]( y, 
                psi_function = pointer_to_psi_function )
        # Accumulate errors in the standard parameters
        b_error[estimator_index] = (beta_hat -  beta)**2
        g_error[estimator_index] = (gamma_hat - gamma)**2
        a_error[estimator_index] = (alpha_hat - alpha)**2
        # Get estimated physical parameters 
        f_d_hat, phase_s_hat, rho_hat = to_physical_parameters( alpha_hat, beta_hat, gamma_hat,
                                                                T_m, T_sampling, delta_0 )
        # Accumulate errors in the physical parameters
        f_error[estimator_index] = (f_d_hat     - f_d_now    )**2
        r_error[estimator_index] = (rho_hat     - rho_now    )**2
        p_error[estimator_index] = (phase_s_hat - phase_s_now)**2

    return [b_error, g_error, a_error, f_error, r_error, p_error]

In [4]:
def MC_experiments( nrof_samples = 500, rho = 2, f_d = 73, phase_s = np.pi, SNR_out = 20, SNR_in = 40,
                    T_sampling = 1e-4, T_m = 1e-8, delta_0 = 5e-6, nrof_MCs = 100, 
                    estimators = [ PCP, grid_search ],
                    randomized = False, rho_range = [1, 2], f_d_range = [-200, 400], 
                    phase_s_range = [0, 2*np.pi] ):
    
    # Initialize pool of workers
    pool = mp.Pool( 3 )
    
    # Convert SNR_in to a standard deviation v_std 
    # (SNR_out will be treated later, since w_std will depend on psi)
    v_std = 10**( - SNR_in / 20 )    
    
    # Get lenght of the grid
    length_of_grid = np.max( [ np.array( rho          ).size, 
                               np.array( f_d          ).size,
                               np.array( phase_s      ).size,
                               np.array( nrof_samples ).size,
                               np.array( SNR_out      ).size,
                               np.array( v_std        ).size ] )
    
    # Initialize error vectors
    # Physical parameters
    f_d_error     = np.zeros( ( len(estimators), length_of_grid ) ) 
    phase_s_error = np.zeros_like( f_d_error )
    rho_error     = np.zeros_like( f_d_error )
    # Standard parameters
    beta_error    = np.zeros_like( f_d_error )
    gamma_error   = np.zeros_like( f_d_error )
    alpha_error   = np.zeros_like( f_d_error )
        
    # Inform user
    print( "Starting Montecarlo runs (%d per grid point), on %d grid points, sit tight..."%( 
            nrof_MCs, length_of_grid ) )
    # Record starting time
    start = time.time( )
    # Set up user info prompt
    print( "grid_point = ", end = '' )
    # For every sample size
    for grid_point in range( length_of_grid ): 
               
        # Inform user
        print( "%d"%( grid_point ), end = ', ' )
        
        # Run parallelized asynchronous MCs
        result_objects = [ pool.apply_async( experiment, 
                           args = ( seed, grid_point, rho, rho_range, f_d, f_d_range, phase_s, 
                                    phase_s_range, nrof_samples, SNR_out, v_std, T_m, T_sampling,
                                    delta_0, estimators, randomized ) 
                           ) for seed in np.random.randint( 0, 1e9, size = (nrof_MCs) ) ]
        # Obtain results from result objects
        results = np.array( [result.get() for result in result_objects] )
        # Average over repetitions and change to dB
        results = 10 * np.log10( np.mean( results, axis = 0 ) )
        # Extract errors for the current grid point in the right positions
        ( beta_error[:,grid_point] , gamma_error[:,grid_point], 
          alpha_error[:,grid_point], f_d_error[:,grid_point]  ,
          rho_error[:,grid_point]  , phase_s_error[:,grid_point] ) = results
    
    # Close and join the pool
    pool.close( )
    pool.join( )
    
    computation_time = time.time( ) - start
    print( "Finished in %d min, %f s."%( computation_time//60, (computation_time%(60)) ) )
    
    # Return mean squared errors [dB] for physical and standard parameters through the grid
    return ( f_d_error, phase_s_error, rho_error, beta_error, gamma_error, alpha_error )